In [3]:
import datetime
import json
import logging
import views
import argparse



# DATASETS is a dictionary of Dataset objects.
from views import DATASETS
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
# These are the building blocks of the modelling interface
from views import Ensemble, Model, Downsampling, Period
# These are model specifications from the specfiles
from views.specs.models import cm as model_specs_cm, pgm as model_specs_pgm
from views.specs.periods import get_periods, get_periods_by_name
# Utils
# from views.utils import db, io, data as datautils
from views.utils.data import assign_into_df
from views.apps.pipeline.models_cm import all_cm_models_by_name
from views.apps.pipeline.models_pgm import all_pgm_models_by_name
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

#logging errors
logging.basicConfig(
    level=logging.DEBUG,
    # level=logging.INFO, # uncomment this and comment debug above for less yelling in red
    format=views.config.LOGFMT,
)


#get periods
run_id = "d_2020_10_01_prelim"
periods = get_periods(run_id) # as a list
periods_by_name = get_periods_by_name(run_id)# as a dict
period_a = periods_by_name["A"]
period_b = periods_by_name["B"]
period_c = periods_by_name["C"]


#bring in dataset
dataset = views.DATASETS["cm_africa_imp_0"]
df = dataset.df


# get models
model_from_pipeline_spec = all_cm_models_by_name["cm_sb_acled_violence"]
model_from_pipeline_spec1 = all_cm_models_by_name["cm_sb_cflong"]
model_from_pipeline_spec2 = all_cm_models_by_name["cm_sb_neibhist"]
model_from_pipeline_spec3 = all_cm_models_by_name["cm_sb_cdummies"]
model_from_pipeline_spec4 = all_cm_models_by_name["cm_sb_acled_protest"]
model_from_pipeline_spec5 = all_cm_models_by_name["cm_sb_reign_coups"]
model_from_pipeline_spec6 = all_cm_models_by_name["cm_sb_icgcw"]
model_from_pipeline_spec7 = all_cm_models_by_name["cm_sb_reign_drought"]
model_from_pipeline_spec8 = all_cm_models_by_name["cm_sb_reign_global"]
model_from_pipeline_spec9 = all_cm_models_by_name["cm_sb_vdem_global"]
model_from_pipeline_spec10 = all_cm_models_by_name["cm_sb_demog"]
model_from_pipeline_spec11= all_cm_models_by_name["cm_sb_wdi_global"]
model_from_pipeline_spec12 = all_cm_models_by_name["cm_sb_all_global"]
model_from_pipeline_spec13 = all_cm_models_by_name["cm_sbonset24_25_all"]


#create array of models
models=[
        model_from_pipeline_spec ,
        model_from_pipeline_spec1,
        model_from_pipeline_spec2,
        model_from_pipeline_spec3 ,
        model_from_pipeline_spec4,
        model_from_pipeline_spec5,
        model_from_pipeline_spec6 ,
        model_from_pipeline_spec7,
        model_from_pipeline_spec8,
        model_from_pipeline_spec9 ,
        model_from_pipeline_spec10,
        model_from_pipeline_spec11,
        model_from_pipeline_spec12,
        model_from_pipeline_spec13
       ]

#update the periods for each of the models
for model in models:
    model.periods = periods

#Build ensemble
cflong_acled_violence_ensemble = Ensemble(
    name="cflong_acled_violence_ensemble",
    models=models,
    outcome_type="prob",
    col_outcome="greq_25_ged_best_sb",
    method="average",
    periods=periods
)
ensembles = [cflong_acled_violence_ensemble]



[2020-12-28 15:50:30,832] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-12-28 15:50:30,850] - views.utils.io:107 - DEBUG - Loading YAML from /home/kyle/code/Views2/OpenViEWS2/views/specs/periods/periods.yaml
[2020-12-28 15:50:30,871] - views.utils.io:65 - DEBUG - Reading parquet at /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet with cols None
[2020-12-28 15:50:31,416] - views.utils.io:72 - DEBUG - Finished reading parquet from /home/kyle/code/Views2/OpenViEWS2/storage/data/datasets/cm_africa_imp_0.parquet.


In [4]:
periods

[Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432),
 Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468),
 Period(name='C', train_start=121, train_end=480, predict_start=490, predict_end=527)]

In [5]:
for model in models:
    model.fit_estimators(df, populate_extras = False)

[2020-12-28 15:51:09,219] - views.apps.model.api:441 - INFO - Fitting estimators for cm_sb_acled_violence
[2020-12-28 15:51:09,219] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_violence for period A step 1
[2020-12-28 15:51:09,229] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_violence
[2020-12-28 15:51:09,230] - views.apps.model.api:422 - DEBUG - cm_sb_acled_violence downsampled away 0
[2020-12-28 15:51:09,233] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_violence on 12399 rows
[2020-12-28 15:51:09,234] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_violence
[2020-12-28 15:51:32,996] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_violence A 1 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_1.joblib
[2020-12-28 15:51:46,865] - views.apps.model.api:119 - DEBUG - cm_sb_acled_violence saved to /home/kyle/code/V

[2020-12-28 15:56:32,057] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_violence
[2020-12-28 15:56:56,441] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_violence A 36 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_36.joblib
[2020-12-28 15:57:08,391] - views.apps.model.api:119 - DEBUG - cm_sb_acled_violence saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_36.joblib
[2020-12-28 15:57:08,413] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_violence for period A step 38
[2020-12-28 15:57:08,420] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_violence
[2020-12-28 15:57:08,421] - views.apps.model.api:422 - DEBUG - cm_sb_acled_violence downsampled away 0
[2020-12-28 15:57:08,421] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_violence on 10541 rows
[2020-12-28 15:57:08,421] - views.apps

[2020-12-28 16:03:17,738] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_violence
[2020-12-28 16:03:17,739] - views.apps.model.api:422 - DEBUG - cm_sb_acled_violence downsampled away 0
[2020-12-28 16:03:17,739] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_violence on 12777 rows
[2020-12-28 16:03:17,740] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_violence
[2020-12-28 16:03:48,211] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_violence B 30 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_B_30.joblib
[2020-12-28 16:04:03,412] - views.apps.model.api:119 - DEBUG - cm_sb_acled_violence saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_B_30.joblib
[2020-12-28 16:04:03,439] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_violence for period B step 36
[2020-12-28 16:04:03,449] - views.apps

[2020-12-28 16:10:48,671] - views.apps.model.api:119 - DEBUG - cm_sb_acled_violence saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_18.joblib
[2020-12-28 16:10:48,693] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_acled_violence for period C step 24
[2020-12-28 16:10:48,702] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_acled_violence
[2020-12-28 16:10:48,703] - views.apps.model.api:422 - DEBUG - cm_sb_acled_violence downsampled away 0
[2020-12-28 16:10:48,703] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_acled_violence on 15693 rows
[2020-12-28 16:10:48,703] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_acled_violence
[2020-12-28 16:11:19,419] - views.apps.model.api:116 - DEBUG - Saving cm_sb_acled_violence C 24 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_24.joblib
[2020-12-28 16:11:38,113] - views.apps

[2020-12-28 16:19:16,541] - views.apps.model.api:116 - DEBUG - Saving cm_sb_cflong A 12 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_A_12.joblib
[2020-12-28 16:19:31,304] - views.apps.model.api:119 - DEBUG - cm_sb_cflong saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_A_12.joblib
[2020-12-28 16:19:31,315] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_cflong for period A step 18
[2020-12-28 16:19:31,389] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_cflong
[2020-12-28 16:19:31,390] - views.apps.model.api:422 - DEBUG - cm_sb_cflong downsampled away 0
[2020-12-28 16:19:31,390] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_cflong on 13932 rows
[2020-12-28 16:19:31,390] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_cflong
[2020-12-28 16:20:19,665] - views.apps.model.api:116 - DEBUG - Saving cm_sb_cflong A 18 to /home/kyle/code/Vie

...

[2020-12-28 23:36:37,675] - views.apps.model.api:119 - DEBUG - cm_sb_all_global saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_C_12.joblib
[2020-12-28 23:36:37,687] - views.apps.model.api:444 - DEBUG - Fitting cm_sb_all_global for period C step 18
[2020-12-28 23:36:37,829] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sb_all_global
[2020-12-28 23:36:37,830] - views.apps.model.api:422 - DEBUG - cm_sb_all_global downsampled away 0
[2020-12-28 23:36:37,830] - views.apps.model.api:424 - DEBUG - Fitting cm_sb_all_global on 15993 rows
[2020-12-28 23:36:37,831] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sb_all_global
[2020-12-28 23:38:22,177] - views.apps.model.api:116 - DEBUG - Saving cm_sb_all_global C 18 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_all_global_C_18.joblib
[2020-12-28 23:38:34,719] - views.apps.model.api:119 - DEBUG - cm_sb_all_g

[2020-12-28 23:48:12,163] - views.apps.model.api:399 - DEBUG - cm_sbonset24_25_all has onset_outcome, transforming outcome
[2020-12-28 23:48:12,294] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sbonset24_25_all
[2020-12-28 23:48:12,295] - views.apps.model.api:422 - DEBUG - cm_sbonset24_25_all downsampled away 0
[2020-12-28 23:48:12,295] - views.apps.model.api:424 - DEBUG - Fitting cm_sbonset24_25_all on 9801 rows
[2020-12-28 23:48:12,295] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sbonset24_25_all
[2020-12-28 23:48:51,479] - views.apps.model.api:116 - DEBUG - Saving cm_sbonset24_25_all A 9 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_A_9.joblib
[2020-12-28 23:48:56,221] - views.apps.model.api:119 - DEBUG - cm_sbonset24_25_all saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_A_9.joblib
[2020-12-28 23:48:56,229] - views.apps.m

[2020-12-28 23:54:04,951] - views.apps.model.api:119 - DEBUG - cm_sbonset24_25_all saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_B_1.joblib
[2020-12-28 23:54:04,960] - views.apps.model.api:444 - DEBUG - Fitting cm_sbonset24_25_all for period B step 3
[2020-12-28 23:54:05,072] - views.apps.model.api:399 - DEBUG - cm_sbonset24_25_all has onset_outcome, transforming outcome
[2020-12-28 23:54:05,212] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sbonset24_25_all
[2020-12-28 23:54:05,213] - views.apps.model.api:422 - DEBUG - cm_sbonset24_25_all downsampled away 0
[2020-12-28 23:54:05,213] - views.apps.model.api:424 - DEBUG - Fitting cm_sbonset24_25_all on 11553 rows
[2020-12-28 23:54:05,213] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sbonset24_25_all
[2020-12-28 23:54:54,351] - views.apps.model.api:116 - DEBUG - Saving cm_sbonset24_25_all B 3 to /home/ky

[2020-12-29 00:00:12,987] - views.apps.model.api:424 - DEBUG - Fitting cm_sbonset24_25_all on 10267 rows
[2020-12-29 00:00:12,987] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sbonset24_25_all
[2020-12-29 00:00:57,324] - views.apps.model.api:116 - DEBUG - Saving cm_sbonset24_25_all B 36 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_B_36.joblib
[2020-12-29 00:01:02,104] - views.apps.model.api:119 - DEBUG - cm_sbonset24_25_all saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_B_36.joblib
[2020-12-29 00:01:02,113] - views.apps.model.api:444 - DEBUG - Fitting cm_sbonset24_25_all for period B step 38
[2020-12-29 00:01:02,227] - views.apps.model.api:399 - DEBUG - cm_sbonset24_25_all has onset_outcome, transforming outcome
[2020-12-29 00:01:02,362] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sbonset24_25_all
[2020-12-29 00:01:02,363] 

[2020-12-29 00:08:21,871] - views.apps.model.api:399 - DEBUG - cm_sbonset24_25_all has onset_outcome, transforming outcome
[2020-12-29 00:08:22,118] - views.apps.model.api:413 - DEBUG - Downsampling by Downsampling(share_positive=1.0, share_negative=1.0, threshold=0) for cm_sbonset24_25_all
[2020-12-29 00:08:22,118] - views.apps.model.api:422 - DEBUG - cm_sbonset24_25_all downsampled away 0
[2020-12-29 00:08:22,119] - views.apps.model.api:424 - DEBUG - Fitting cm_sbonset24_25_all on 12574 rows
[2020-12-29 00:08:22,119] - views.apps.model.api:137 - DEBUG - Getting initial_estimator for cm_sbonset24_25_all
[2020-12-29 00:09:22,821] - views.apps.model.api:116 - DEBUG - Saving cm_sbonset24_25_all C 24 to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_24.joblib
[2020-12-29 00:09:28,128] - views.apps.model.api:119 - DEBUG - cm_sbonset24_25_all saved to /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sbonset24_25_all_C_24.joblib
[2020-12-29 00:09:28,136] - views.ap

In [2]:
for model in models:
    # Uncalibrated predictions
    df_pred = model.predict(df)
    df = assign_into_df(df_to=df, df_from=df_pred)

    df_pred = model.predict_calibrated(
        df=df.fillna(0),
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)

for ensemble in ensembles:
    df_pred = ensemble.predict(
        df=df.fillna(0),
        period_calib=period_b,
        period_test=period_c,
    )
    df = assign_into_df(df_to=df, df_from=df_pred)

[2020-12-28 15:23:17,205] - views.apps.model.api:552 - INFO - Predicting for cm_sb_acled_violence
[2020-12-28 15:23:17,206] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_violence periods: [Period(name='A', train_start=121, train_end=396, predict_start=397, predict_end=432), Period(name='B', train_start=121, train_end=432, predict_start=433, predict_end=468), Period(name='C', train_start=121, train_end=480, predict_start=490, predict_end=527)]
[2020-12-28 15:23:17,237] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_1.joblib
[2020-12-28 15:23:24,416] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_3.joblib
[2020-12-28 15:23:31,759] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_A_6.joblib
[2020-12-28 15:23:38,399] - views.apps.model.api:125 - DEBUG - Loading /home/ky

[2020-12-28 15:28:55,495] - views.apps.model.api:553 - DEBUG - Predicting for cm_sb_acled_violence periods: [Period(name='C', train_start=121, train_end=480, predict_start=490, predict_end=527)]
[2020-12-28 15:28:55,534] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_1.joblib
[2020-12-28 15:29:05,170] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_3.joblib
[2020-12-28 15:29:14,621] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_6.joblib
[2020-12-28 15:29:23,907] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_9.joblib
[2020-12-28 15:29:32,442] - views.apps.model.api:125 - DEBUG - Loading /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_acled_violence_C_12.joblib
[2020-12-28 15:29:41,306] - views.apps.model

RuntimeError: No estimator stored at /home/kyle/code/Views2/OpenViEWS2/storage/models/cm_sb_cflong_A_1.joblib

In [ ]:



cols_predict = [ensemble.col_sc_calibrated for ensemble in ensembles]
df_results = df.loc[period_c.times_predict, cols_predict]

def createList(r1, r2):
    return list(range(r1, r2+1))


if int(args.start_date) < int(period_c.times_predict[0]):
    args.start_date = period_c.times_predict[0]
elif int(args.start_date) > int(max(period_c.times_predict)):
    args.start_date = int(max(period_c.times_predict))

if int(args.end_date) < int(args.start_date):
    args.end_date = int(args.start_date) + 1
if int(args.end_date) > int(max(period_c.times_predict)):
    args.end_date = max(period_c.times_predict)



time_p = createList(int(args.start_date), int(args.end_date))

results = df_results.loc[time_p]

nameofcsv_test = "storage/predictions/views_sb_ensemble_script_start=" + str(args.start_date) +"_end="+str(args.end_date) +"_currentTime="+ str(datetime.datetime.now()) + ".csv"
results.to_csv(nameofcsv_test)
